In [62]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import ast
import itertools

import iqplot

import bokeh.io
import bokeh.models
import bokeh.plotting
import ast
import holoviews as hv
import colorcet
bokeh.io.output_notebook()
hv.extension('bokeh')

In [40]:
Data = pd.read_csv('/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiSensory_Project/GatedArenas/Results/DataSetAugust22.csv')
Data.head()

In [41]:
Data_noWater = Data[Data['Test Starvation'] == "Overnight no Water"]
Data_noWater_Simple = Data_noWater[Data_noWater["Training Starvation"] == "Not starved"]#.reset_index()

In [42]:
Data_noWater_Simple["Peeks Left"] = [160] * len(Data_noWater_Simple)

In [43]:
param = 160

for index, row in Data_noWater_Simple.iterrows():
    print (row['Durations Left Corner'])

    #print (1 for i in row['Durations Left Corner'])
    Data_noWater_Simple.loc[index,"Peeks Left"] = sum(1 for i in ast.literal_eval(row['Durations Left Corner']) if i > param)
    #Data_noWater_Simple['Peeks Left'][rows]= sum(1 for i in Data['Durations Left Corner'][rows] if i > param)


       # Data_noWater_Simple.apply(sum(1 for i in Data['Durations Left Corner'] if i > param) for elements in Data['Durations Left Corner'])
Data_noWater_Simple['Peeks Left']

In [7]:
p_box = iqplot.stripbox(data=Data_noWater_Simple,
                   q="Peeks Left",
                   cats="Training",
                   )

bokeh.io.show(p_box)



In [74]:
source = bokeh.models.ColumnDataSource(
    dict(
        Training=Data_noWater_Simple["Training"],
        LCDurations=Data_noWater_Simple["Durations Left Corner"],
        LPeeks=Data_noWater_Simple["Peeks Left"],
    )
)

In [75]:
s_box = iqplot.stripbox(data=source,
                   q=source.data["LPeeks"],
                   cats=source.data["Training"],
                   )

bokeh.io.show(s_box)

In [63]:
def no_ygrid_hook(plot, element):
    """Hook for disabling x-grid lines."""
    plot.handles["plot"].ygrid.grid_line_color = None

boxwhisker_opts = dict(
    box_color="origin",
    box_line_color="white",
    box_line_width=1.5,
    box_width=0.4,
    cmap=colorcet.b_glasbey_category10,
    frame_height=150,
    frame_width=400,
    hooks=[no_ygrid_hook],
    invert_axes=True,
    invert_yaxis=True,
    outlier_line_alpha=0,
    show_grid=True,
    title="HoloViews",
)


In [65]:
import bokeh.sampledata.autompg
df = bokeh.sampledata.autompg.autompg_clean
df['cyl'] = df['cyl'].astype(str)

testhv = hv.BoxWhisker(data=df, kdims="origin", vdims="mpg",).opts(**boxwhisker_opts)

bokeh.io.show(testhv)

In [67]:
phv = hv.BoxWhisker(data=Data_noWater_Simple,
                    kdims="Training",
                    vdims="Peeks Left")#.opts(**boxwhisker_opts)

bokeh.io.show(hv.render(phv))

In [48]:
lay = bokeh.layouts.row(p_box,p_box)
bokeh.io.show(lay)

In [8]:
row = 10
Data_noWater_Simple['Durations Left Corner'][row]

In [44]:
source = bokeh.models.ColumnDataSource(dict(Training=Data_noWater_Simple["Training"],
                                            LCDurations= Data_noWater_Simple['Durations Left Corner'],
                                            LPeeks=Data_noWater_Simple['Peeks Left']))

In [46]:
p_box = iqplot.stripbox(source=source,
                   q="LPeeks",
                   cats="Training",
                   )

bokeh.io.show(p_box)

In [10]:
source.data['LPeeks']

In [21]:
Data_noWater_Simple["Durations Left Corner"]

In [35]:


Durs = source.data['LCDurations']
Durs = Durs.values

In [33]:

#Durs[0] = ast.literal_eval(Durs[0])
for i in Durs:
    Durs[i] = ast.literal_eval(Durs[i])

In [34]:

#for i in itertools.chain.from_iterable(Durs):

    #print(ast.literal_eval(Durs[i]))

Durs

In [12]:
param = 40
for j in Durs:
    #print(source.data['LCDurations'][row])
    LPeeks = sum(1 for i in ast.literal_eval(Durs[j]) if i > param)
    #source.data.loc["LPeeks"] = sum(1 for i in ast.literal_eval(source.data['LCDurations'][row]) if i > param)

#source.data["LPeeks"]

In [12]:
Melted = pd.melt(Data_noWater_Simple,
                 id_vars=['Training', 'ObjectsReinforced','Reinforced_side',
                          'Fly', 'Date'],
                 value_name = 'Count',
                 value_vars=['Peeks Left', 'Peeks Right', 'Peeks Top', 'Long Peeks Left', 'Long Peeks Right', 'Long Peeks Top','Face Left', 'Face Right', 'Face Top'],
                 var_name='Type')

In [11]:
conditions = [
    (Melted['Reinforced_side'].str.contains('Right') & (Melted['Type'].str.contains('Right'))),
    (Melted['Reinforced_side'].str.contains('Left') & (Melted['Type'].str.contains('Left'))),
    (Melted['Reinforced_side'].str.contains('Right') & (Melted['Type'].str.contains('Left'))),
    (Melted['Reinforced_side'].str.contains('Left') & (Melted['Type'].str.contains('Right'))),
    (Melted['Type'].str.contains('Top')),
]

values = ['Rewarded Side', 'Rewarded Side', 'Punished Side', 'Punished Side', 'Empty Side']
Melted['Condition'] = np.select(conditions,values)
Melted.head()

In [16]:
conditions = [
    (Melted['Type'].str.contains('Long')),
    (Melted['Type'].str.contains('Face')),
    (('Long' not in Melted['Type']) & ('Face' not in Melted['Type'])) # This is not really working but still gives expected result
]

values = ['Squeeze', 'Fronts', 'Peeks']
Melted['Beh'] = np.select(conditions,values)
Melted.head()

In [17]:
sns.set(font_scale=1)
g=sns.catplot(x='Condition', y = 'Count',
            hue='Training', showfliers = False,
            col='Beh',
            kind="box",
            row='ObjectsReinforced',
            data= Melted)
g.map_dataframe(sns.stripplot,x='Condition', y = 'Count',
            hue='Training', dodge=True,
            palette=["#404040"])
plt.show

#plt.savefig('/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiSensory_Project/GatedArenas/Results/220722_BehaviorPlot_ObjectSplit.png')

In [8]:
Melted.Training = pd.Categorical(Melted.Training,
                                 categories=['Trained', 'Ctrl'])

In [67]:
TimeData = pd.read_csv('/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiSensory_Project/GatedArenas/Results/DataSetNew.csv')

TimeData.head()

In [68]:
TimeMelted = pd.melt(TimeData, id_vars=['Training', 'ObjectsReinforced',
                                        'Reinforced_side', 'Date', 'Fly'],
                     value_name = 'Values',
                     value_vars=['Visits Left Corner', 'Durations Left Corner',
          'Visits Right Corner', 'Durations Right Corner',
          'Visits Top Corner', 'Durations Top Corner',
          'Visits Left Front', 'Durations Left Front',
          'Visits Right Front', 'Durations Right Front',
          'Visits Top Front', 'Durations Top Front'],
                     var_name="Variable")

TimeMelted[['Values',]] = TimeMelted[['Values',]].applymap(ast.literal_eval)
TimeMelted = TimeMelted.explode('Values').drop_duplicates()

conditions = [
    (TimeMelted['Variable'].str.contains('Durations')),
    (TimeMelted['Variable'].str.contains('Visits')),
]

values = ['Durations', 'Visits']
TimeMelted['Kind'] = np.select(conditions,values)
TimeMelted.head()

conditions = [
    (TimeMelted['Reinforced_side'].str.contains('Right') & (TimeMelted['Variable'].str.contains('Right'))),
    (TimeMelted['Reinforced_side'].str.contains('Left') & (TimeMelted['Variable'].str.contains('Left'))),
    (TimeMelted['Reinforced_side'].str.contains('Right') & (TimeMelted['Variable'].str.contains('Left'))),
    (TimeMelted['Reinforced_side'].str.contains('Left') & (TimeMelted['Variable'].str.contains('Right'))),
    (TimeMelted['Variable'].str.contains('Top')),
]

values = ['Rewarded Side', 'Rewarded Side', 'Punished Side', 'Punished Side', 'Empty Side']
TimeMelted['Condition'] = np.select(conditions,values)
TimeMelted.head()

,Training,ObjectsReinforced,Reinforced_side,Date,Fly,Variable,Values,Kind,Condition
0,Ctrl,Orange,Left,22-03-10,1,Visits Left Corner,NaN,Visits,Rewarded Side
1,Ctrl,Orange,Left,22-03-10,2,Visits Left Corner,452,Visits,Rewarded Side
1,Ctrl,Orange,Left,22-03-10,2,Visits Left Corner,2689,Visits,Rewarded Side
1,Ctrl,Orange,Left,22-03-10,2,Visits Left Corner,4476,Visits,Rewarded Side
1,Ctrl,Orange,Left,22-03-10,2,Visits Left Corner,4538,Visits,Rewarded Side


In [11]:
ReinforcedVisit = TimeMelted[TimeMelted['Condition'] == "Rewarded Side"]

ReinforcedVisit.loc[ReinforcedVisit['Variable'].str.contains('Visits'), 'Variable'] = 'Visits'
ReinforcedVisit.loc[ReinforcedVisit['Variable'].str.contains('Durations'), 'Variable'] = 'Durations'

ReinforcedVisit = ReinforcedVisit.reset_index().pivot_table(index = ['Fly', 'Training', 'ObjectsReinforced'],
                                          columns = 'Variable',
                                          values = 'Values').reset_index()

In [12]:
PunishedVisits = TimeMelted[TimeMelted['Condition'] == "Punished Side"]

PunishedVisits.loc[PunishedVisits['Variable'].str.contains('Visits'), 'Variable'] = 'Visits'
PunishedVisits.loc[PunishedVisits['Variable'].str.contains('Durations'), 'Variable'] = 'Durations'

PunishedVisits = PunishedVisits.reset_index().pivot_table(index = ['Fly', 'Training', 'ObjectsReinforced'],
                                          columns = 'Variable',
                                          values = 'Values').reset_index()

In [13]:
# Create the figure, stored in variable `p`
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label='Visits time',
    y_axis_label='Visits duration',
    x_range=[0, 48000]
)


p.circle(
    source= ReinforcedVisit[ReinforcedVisit['Training']== 'Trained'],
    x='Visits',
    y='Durations',
    legend_label = 'Trained flies',

)

p.circle(
    source=ReinforcedVisit[ReinforcedVisit['Training']== 'Ctrl'],
    x='Visits',
    y='Durations',
    color = 'orange',
    legend_label = 'Control flies'
)

bokeh.io.show(p)

In [14]:
# Create the figure, stored in variable `p`
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label='Visits time',
    y_axis_label='Visits duration',
    x_range=[0, 48000]
)


p.circle(
    source= PunishedVisits[PunishedVisits['Training']== 'Trained'],
    x='Visits',
    y='Durations',
    legend_label = 'Trained flies'
)

p.circle(
    source=PunishedVisits[PunishedVisits['Training']== 'Ctrl'],
    x='Visits',
    y='Durations',
    color = 'orange',
    legend_label = 'Control flies'
)

bokeh.io.show(p)

In [15]:
EmptyVisits = TimeMelted[TimeMelted['Condition'] == "Empty Side"]

EmptyVisits.loc[EmptyVisits['Variable'].str.contains('Visits'), 'Variable'] = 'Visits'
EmptyVisits.loc[EmptyVisits['Variable'].str.contains('Durations'), 'Variable'] = 'Durations'

EmptyVisits = EmptyVisits.reset_index().pivot_table(index = ['Fly', 'Training', 'ObjectsReinforced'],
                                          columns = 'Variable',
                                          values = 'Values').reset_index()

In [16]:
# Create the figure, stored in variable `p`
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label='Visits time',
    y_axis_label='Visits duration',
    x_range=[0, 48000]
)


p.circle(
    source= EmptyVisits[EmptyVisits['Training']== 'Trained'],
    x='Visits',
    y='Durations',
    legend_label = 'Trained flies'
)

p.circle(
    source=EmptyVisits[EmptyVisits['Training']== 'Ctrl'],
    x='Visits',
    y='Durations',
    color = 'orange',
    legend_label = 'Control flies'
)

bokeh.io.show(p)

In [17]:
RightCornerVisits = TimeMelted[TimeMelted['Variable'].isin(['Visits Right Corner', 'Durations Right Corner'])].reset_index().pivot_table(index = ['Fly', 'Training'], columns = 'Variable', values = 'Values').reset_index()

# Create the figure, stored in variable `p`
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label='Visits time',
    y_axis_label='Visits duration'
)


p.circle(
    source= RightCornerVisits[RightCornerVisits['Training']== 'Trained'],
    x='Visits Right Corner',
    y='Durations Right Corner',
    legend_label = 'Trained flies'
)

p.circle(
    source=RightCornerVisits[RightCornerVisits['Training']== 'Ctrl'],
    x='Visits Right Corner',
    y='Durations Right Corner',
    color = 'orange',
    legend_label = 'Control flies'
)

bokeh.io.show(p)

Bokeh boxplots

In [19]:

ThreshSlider = bokeh.models.Slider(
    title="Peeks",
    start=0,
    end=600,
    step=10,
    value=160
)


def slider_callback(attr, old, new):
    param = ThreshSlider.value

    Data['Peekings_Left'] = sum(1 for i in Data['Durations_Corner_Left'] if i > param)
    #Peeks_Right = sum(1 for i in Durations_Corner_Right if i > 160)
    #Peeks_Top = sum(1 for i in Durations_Corner_Top if i > 160)

slider.on_change("value", slider_callback)

p_box = iqplot.stripbox(data=Data,
                   q="Peekings_Left",
                   cats="Training",
                   )

Box_layout = bokeh.layouts.row(
    p_box,
    bokeh.models.Spacer(width=15),
    bokeh.layouts.column(
        ThreshSlider,
        width=200,
    ),
)

def induction_app(doc):
    doc.add_root(induction_layout)

In [6]:
param = 160

Data['Peekings_Left'] = sum(1 for i in Data['Durations Left Corner'] if i > param)


TypeError: '>' not supported between instances of 'str' and 'int'